In [60]:
import pandas as pd

In [61]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [94]:
heart_dis_df = pd.read_csv("processed.cleveland.csv")

In [95]:
heart_dis_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [96]:
heart_dis_df.drop(heart_dis_df[heart_dis_df['ca']=="?"].index, inplace=True)
heart_dis_df.drop(heart_dis_df[heart_dis_df['thal']=="?"].index, inplace=True)

In [97]:
heart_dis_df["ca"] = heart_dis_df["ca"].astype('float32')
heart_dis_df["thal"] = heart_dis_df["thal"].astype('float32')

In [71]:
heart_dis_df = pd.get_dummies(heart_dis_df, columns=["cp", "ca"])

In [85]:
heart_dis_df["sex"] = heart_dis_df["sex"].astype("int8")
heart_dis_df["sex"] = heart_dis_df["ca"].astype("int8")

In [82]:
heart_dis_df.head()

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,thal,target,cp_1.0,cp_2.0,cp_3.0,cp_4.0,ca_0.0,ca_1.0,ca_2.0,ca_3.0
0,63.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,6.0,0,1,0,0,0,1,0,0,0
1,67.0,1.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,2,0,0,0,1,0,0,0,1
2,67.0,1.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,7.0,1,0,0,0,1,0,0,1,0
3,37.0,1.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,3.0,0,0,0,1,0,1,0,0,0
4,41.0,0.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,3.0,0,0,1,0,0,1,0,0,0


In [73]:
X = heart_dis_df.drop(["target"], axis=1).values
y = heart_dis_df.target.values


In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
estimators = [
    ("lr", LogisticRegression()),
    ('svc', make_pipeline(StandardScaler(), SVC())),
    ("rf", RandomForestClassifier())
]

stacked_clf = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier())

In [80]:
stacked_clf.fit(X_train, y_train)

/home/ashkan/Documents/ML3/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ashkan/Documents/ML3/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('svc',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('svc', SVC())])),
                               ('rf', RandomForestClassifier())],
                   final_estimator=GradientBoostingClassifier())

In [81]:
stacked_clf.score(X_test, y_test)

0.6166666666666667

In [84]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train).score(X_test, y_test)

0.6